# DCGAN 
###  this network is the same architecture as gans but with some enhancements and changes so I followed this architecture but I used kernal size 4 to be able to get the suitable output shape 

![DCGAN](images/DCGAN.png)

In [2]:
import torch 
from torch import nn 
import torch.optim as optim
import numpy as np 
import sys
from PIL import Image
from tensorflow.keras.datasets import mnist
import os 
import matplotlib.pyplot as plt
import math 
import torchvision.datasets  as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [3]:
class Generator(nn.Module) : 
    def __init__(self) : 
        super().__init__() 
        self.linear_1 = nn.Linear(100 , 7*7*128 ) 
        self.seq = nn.Sequential(
            nn.BatchNorm2d(128) , 
            nn.ReLU() , 
            nn.ConvTranspose2d(128 , 128 , 4 , padding = 1 , stride = 2 ) , 
            nn.BatchNorm2d(128) , 
            nn.ReLU() , 
            nn.ConvTranspose2d(128 , 64 , 4 , padding = 1 , stride = 2 ) , 
            nn.BatchNorm2d(64) , 
            nn.ReLU() , 
            nn.ConvTranspose2d(64 , 32 , 4 , padding = 1 , stride = 1 ) , 
            nn.BatchNorm2d(32) , 
            nn.ReLU() , 
            nn.ConvTranspose2d(32 , 1 , 4 , padding = 2 , stride = 1 ) , 
            nn.Sigmoid()
        
        )
    def forward(self , X) :
        X = self.linear_1(X) 
        X = X.view(-1 , 128 , 7 , 7 )
        return self.seq(X) 

In [ ]:
class Discriminator(nn.Module) : 
    def __init__(self) : 
        super().__init__() 
        self.seq = nn.Sequential(
            nn.LeakyReLU(.2) , 
            nn.Conv2d(1 , 32 , 4 , padding = 1 , stride = 2 ) ,
            nn.LeakyReLU(.2) , 
            nn.Conv2d(32 , 64 , 4 , padding = 1 , stride = 2 ) ,
            nn.LeakyReLU(.2) , 
            nn.Conv2d(64 , 128 , 4 , padding = 1 , stride = 2 ) ,
            nn.LeakyReLU(.2) , 
            nn.Conv2d(128 , 256 , 4 , padding = 1 , stride = 1 ) ,
            nn.Flatten() , 
            nn.Linear(2 * 2 * 256 , 1)  , 
            nn.Sigmoid() 
            
            
        )
    def forward(self , X ) : 
        return self.seq(X)

In [1]:
def train(epochs ) :  
    loss_fn = nn.BCELoss()
    discriminator = Discriminator() 
    dis_optimizer = optim.Adam(discriminator.parameters(), lr= 2e-4 )
    generator = Generator() 
    gen_optimizer = optim.Adam(generator.parameters(), lr= 2e-4 ) 
    generator.train() 
    discriminator.train() 
    
    
    transforms_ = transforms.Compose(
        [
        transforms.Resize((28 ,28 )),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(1)], [0.5 for _ in range(1)]
        ),
        ]
    )
    dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms_,
                       download=True)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    
    for epoch in range(epochs ) : 
        gen_losses = []
        dis_losses = []
        for batch_images ,_  in dataloader : 
            noise = torch.randn(64 , 100 )
            fake_images = generator(noise) 
            fake_preds = discriminator(fake_images.detach()).reshape(-1)
            real_preds = discriminator(batch_images).reshape(-1)
            
            dis_fake_loss= loss_fn(fake_preds , torch.zeros_like(fake_preds))
            dis_real_loss = loss_fn(real_preds , torch.ones_like(real_preds))
            dis_loss = dis_fake_loss + dis_real_loss 
            dis_losses.append(dis_loss.detach().numpy()) 
            
            discriminator.zero_grad()
            dis_loss.backward()
            dis_optimizer.step() 
            
            output = discriminator((fake_images , fake_labels).reshape(-1)
            generator_loss = loss_fn(output , torch.ones_like(output)) 
            gen_losses.append(generator_loss.detach().numpy()) 
            
            generator.zero_grad()
            generator_loss.backward()
            gen_optimizer.step()
            
        plot_images(generator,
                torch.randn(16 , 100),
                show=False,
                step=epoch,
                model_name="Dgan")
        print(f'gen_loss:{np.mean(gen_losses)} , dis_loss:{np.mean(dis_losses)}')
                 
    
        
        
        
        
        

SyntaxError: '(' was never closed (3204267025.py, line 43)

In [4]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    
    generator.eval()
    with torch.no_grad() : 
        images = generator((noise , noise_class)) 
    images = images.view(-1  , 28 , 28,1 )
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

In [ ]:
train(10)